In [1]:
from junno import log
log

<IPython.core.display.Javascript object>

#### Import

In [2]:
import liv4dcli as cli
cli.config.url = 'http://206.12.89.104:5050'

In [3]:
import junno.datasets as D
import numpy as np

In [4]:
from skimage.measure import label
from scipy.ndimage.measurements import center_of_mass
import skimage.morphology as skmorph

#### Constante

In [19]:
OUT = '/home/gaby/These/Data/Fundus/MESSIDOR/AV_label_val/'

### Info

In [6]:
_ = cli.user.list_user(True)

User table
 id                                   | name                  | email                            | role      |
 597c679a-fd6e-4c87-833d-26d1d9232a9a | prod                  | prod@prod.com                    | admin     |
 a8d42b15-4d1e-4a95-a95e-852b9712fc71 | demo                  | demo@demo.ca                     | clinician |
 e3252de6-e128-4553-91b7-ed392529cc7b | clement.playout       | clement.playout@polymtl.ca       | admin     |
 de6f4028-bc85-4bf1-9a87-3e9467cf4383 | farida.cheriet        | farida.cheriet@polymtl.ca        | admin     |
 9b869c98-d334-426a-a824-7ca80c50e344 | philippe.debanne      | philippe.debanne@polymtl.ca      | admin     |
 ed2f771f-fd43-4f0f-98f3-c42edfb5f3b4 | ananda.kalevar        | ananda.kalevar@usherbrooke.ca    | clinician |
 84be2a52-5491-4d67-ad75-1704964eeb51 | mariecarole           | mariecarole@gmail.com            | clinician |
 de7b71eb-87b0-4dce-84fe-373e86dc7eb8 | wongd                 | wongd@smh.ca                     | cl

In [7]:
me = "b45c99c9-b10e-4cd9-96d7-4060a889184b"

In [10]:
tasks = cli.task.list_task(me, True)
imgs = [t['image']['id'] for t in tasks if t['completed']]

Tasks table
 id   | user                  | img  | active | completed |
  647 | gabriel.lepetit-aimon |    3 |   X    |     X     |
  648 | gabriel.lepetit-aimon |    4 |   X    |     X     |
  649 | gabriel.lepetit-aimon |    5 |   X    |     X     |
  650 | gabriel.lepetit-aimon |    6 |   X    |     X     |
  651 | gabriel.lepetit-aimon |    7 |   X    |     X     |
  652 | gabriel.lepetit-aimon |    8 |   X    |     X     |
  653 | gabriel.lepetit-aimon |    9 |   X    |     X     |
  654 | gabriel.lepetit-aimon |   10 |   X    |     X     |
  655 | gabriel.lepetit-aimon |   11 |   X    |     X     |
  656 | gabriel.lepetit-aimon |   12 |   X    |     X     |
  657 | gabriel.lepetit-aimon |   13 |   X    |     X     |
  658 | gabriel.lepetit-aimon |   14 |   X    |     X     |
  659 | gabriel.lepetit-aimon |   15 |   X    |     X     |
  660 | gabriel.lepetit-aimon |   16 |   X    |     X     |
  661 | gabriel.lepetit-aimon |   17 |   X    |     X     |
  663 | gabriel.lepetit-aimo

### Download & Build dataset

In [20]:
data = {'img_id': [], 'name': []}
biomarkers = {'A':'Arteries', 'V': 'Veins', 'U': 'Vessels - Uncertain'}
for img in log.iter(l):
    r = cli.revision.get_revision(me, img)
    img_info = cli.image.get_image(img)
    data['img_id'].append(img)
    data['name'].append(img_info['path'][8:])
    for k, b in biomarkers.items():
        if k not in data:
            data[k] = []
        data[k].append(cli.revision.export_biomarker(r['svg'], b))
data = {k: np.stack(v) for k, v in data.items()}

In [21]:
raw = D.from_numpy(**data)

### Process

In [23]:
def small_elements(array, max_size=100):
    array = label(array)
    r = np.zeros_like(array)
    j = 0
    for i in range(1, array.max()):
        i_map = array==i
        if i_map.sum() < max_size:
            j+=1
            r[i_map] = j
    return r, j

In [25]:
def joinAVU(A, V, U):
    a, nA = small_elements(A)
    v, nV = small_elements(V)
    u, nU = small_elements(U)
    
    h, w = A.shape
    
    for i in range(1, nU):
        e = u==i
        U[e] = False
        
        centroid = np.ceil(center_of_mass(e))
        c_y = int(centroid[0])
        c_x = int(centroid[1])
        
        a_neighbour = A[max(c_y-10,0):min(c_y+10,h), max(c_x-10,0):min(c_x+10,w)].sum()
        v_neighbour = V[max(c_y-10,0):min(c_y+10,h), max(c_x-10,0):min(c_x+10,w)].sum()
        
        if a_neighbour and a_neighbour>v_neighbour:
            A[e] = True
        if v_neighbour and a_neighbour<v_neighbour:
            V[e] = True
            
    for i in range(1, nA):
        e = a==i
        
        centroid = np.ceil(center_of_mass(e))
        c_y = int(centroid[0])
        c_x = int(centroid[1])
        
        a_neighbour = A[max(c_y-10,0):min(c_y+10,h), max(c_x-10,0):min(c_x+10,w)].sum()
        v_neighbour = V[max(c_y-10,0):min(c_y+10,h), max(c_x-10,0):min(c_x+10,w)].sum()
        
        if a_neighbour and a_neighbour>v_neighbour:
            continue
        elif v_neighbour and a_neighbour<v_neighbour:
            V[e] = True
        A[e] = False
    
    for i in range(1, nV):
        e = v==i
        
        centroid = np.ceil(center_of_mass(e))
        c_y = int(centroid[0])
        c_x = int(centroid[1])
        
        a_neighbour = A[max(c_y-10,0):min(c_y+10,h), max(c_x-10,0):min(c_x+10,w)].sum()
        v_neighbour = V[max(c_y-10,0):min(c_y+10,h), max(c_x-10,0):min(c_x+10,w)].sum()
        
        if v_neighbour and v_neighbour>a_neighbour:
            continue
        elif a_neighbour and v_neighbour<a_neighbour:
            A[e] = True
        V[e] = False
        
    elem = skmorph.disk(radius=4, dtype=np.bool)
    A = skmorph.binary_closing(A, elem)
    V = skmorph.binary_closing(V, elem)
    U = skmorph.binary_closing(U, elem)
    
    r = np.zeros(A.shape, dtype=np.int32)
    r[A] = 1
    r[V] = 2
    r[U] = 3
    return r

dataset = raw.apply('AVU', joinAVU, format=[{1:'red', 2:'blue', 3:'green'}]).apply('name', lambda n: n[:-4])

### Export

In [28]:
dataset.export_files(path=OUT, columns='AVU', ncore=4)